In [1]:
import pandas as pd
import glob as gb
import os

files = gb.glob('./data/Ward_Age/*')
dataname = '町丁目名年齢年代別人口分布'

d=os.getcwd()
prefix = d + '/../data/Ward_Age/'


# c
#g_code = pd.read_csv(prefix+'東京都人口世帯数_estat.txt', encoding='SHIFT-JIS', dtype={'KEY_CODE':str})
#g_code[g_code['HYOSYO']>3]
#g_code=g_code[['KEY_CODE','NAME','CITYNAME']]

c_code = pd.read_csv(prefix + '13_2020.csv', encoding='utf-8', dtype={'大字町丁目コード':str})[['大字町丁目名','大字町丁目コード']]

## 1. 大田区

In [78]:
ohta_file = prefix + '大田区2022年1月_rev.xlsx'

ohta_data = pd.read_excel(ohta_file).set_index('年齢')

ohta_finedata = pd.DataFrame(ohta_data.stack()).reset_index().set_axis(['年齢','町丁目名','人口'],axis=1)

ohta_finedata = ohta_finedata[['町丁目名','年齢','人口']]

ohta_finedata['町丁目名'] = ohta_finedata['町丁目名'].str.replace('　','')

ohta_finedata = pd.merge(ohta_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

ohta_finedata['区']='大田'

In [4]:
#ohta_finedata.to_excel(prefix + dataname + '（大田）.xlsx', index=False)

bin_band =  list(range(0,131,5))
ew=pd.cut(ohta_finedata['年齢'], bin_band, right=False)
#pd.DataFrame(pd.DataFrame(ew).value_counts()).sort_values('年齢')

## 2. 品川

In [79]:
male=list(range(0,241,2))
female=list(range(1,241,2))

shina_file = prefix + '品川区2023年1月_rev.xls'
shina_data = pd.read_excel(shina_file, dtype={'行政コード':int})

shina_m = shina_data.loc[:,['行政コード','丁目名']+male].set_index(['行政コード','丁目名']).stack()
shina_f = shina_data.loc[:,['行政コード','丁目名']+female].set_index(['行政コード','丁目名']).stack()

shina_m_fine= pd.DataFrame(shina_m).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_m_fine['年齢']=shina_m_fine['age']//2

shina_f_fine= pd.DataFrame(shina_f).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_f_fine['年齢']=(shina_f_fine['age'])//2

scols=['丁目名','年齢','人口']
mer=['丁目名','年齢']

shina_m_fine=shina_m_fine.drop(columns=['age'])[scols]
shina_f_fine=shina_f_fine.drop(columns=['age'])[scols]


shina_finedata = pd.merge(shina_m_fine, shina_f_fine, on=mer)

#for t,i in shina_finedata.iterrows():
#    shina_finedata.loc[i,'丁目名'] = shina_finedata.loc[i, '丁目名'].replace('\u3000','')
#    print(shina_finedata.loc[i, '丁目名'])

shina_finedata['人口'] = shina_finedata['人口_x']+shina_finedata['人口_y']
shina_finedata = shina_finedata.drop(columns=['人口_x','人口_y']).reset_index(drop=True)
shina_finedata = shina_finedata.rename(columns={'丁目名':'町丁目名'})
shina_finedata = pd.merge(shina_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

shina_finedata['区']='品川'

In [18]:
shina_finedata

,町丁目名,年齢,人口,大字町丁目名,大字町丁目コード
0,北品川１丁目,0,20,北品川１丁目,131090019001
1,北品川１丁目,1,17,北品川１丁目,131090019001
2,北品川１丁目,2,21,北品川１丁目,131090019001
3,北品川１丁目,3,18,北品川１丁目,131090019001
4,北品川１丁目,4,14,北品川１丁目,131090019001
...,...,...,...,...,...
16195,東八潮,115,0,東八潮,131090005000
16196,東八潮,116,0,東八潮,131090005000
16197,東八潮,117,0,東八潮,131090005000
16198,東八潮,118,0,東八潮,131090005000


In [7]:
#shina_finedata.to_excel(prefix + dataname + '（品川）.xlsx', index=False)

## 3. 目黒

In [80]:
meg_file = prefix +'目黒区2022年12月_rev.xlsx'
meg_data = pd.read_excel(meg_file)
#meg_data

top = meg_data[meg_data['a']=='町丁別年齢別人口表'].reset_index()['index']
end = meg_data[meg_data['a']=='104以上'].reset_index()['index']
#print(list(top))
#print(list(end))

meg_tmp = pd.DataFrame()

#meg_data['町丁目名']=''

for k in top:
    chomei = meg_data.iloc[k+1,0].replace('　','')

    for j in list(range(k+4,k+129)):
        meg_data.loc[j,'町丁目名'] = chomei

    chodf = meg_data.iloc[(k+4):(k+129),:]
    
    chodf = chodf[~(chodf['a'].str.contains('～',na=False))]
    chodf = chodf[~(chodf['a'].str.contains('以上',na=False))]
    
    chodf = chodf.rename(columns={'a':'年齢', 'b':'人口'})
    chodf['年齢'] = chodf['年齢'].astype(int)
    
    meg_tmp = pd.concat([meg_tmp, chodf])

meg_finedata = meg_tmp[['町丁目名','年齢','人口']]
meg_finedata = pd.merge(meg_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

meg_finedata['区']='目黒'

In [9]:
#meg_finedata.to_excel(prefix + dataname + '（目黒）.xlsx', index=False)

## 4. 港区

In [10]:
import tabula

mina_file = prefix + '港区2022年4月.pdf'

mina_data = tabula.read_pdf(mina_file,pages="1-117",stream=True)

In [11]:
textfile = open(prefix+'港区町丁目名.txt','r')
output_text = textfile.read()
textfile.close()

import re

pattern = '(.+丁目)|(麻布.+町)|(麻布狸穴町)$'

minacho = re.findall(pattern, output_text)

from itertools import chain
def flatten(nested_list):
    return list(chain.from_iterable(nested_list))

#flatten(list(map(lambda x: list(x), minacho)))

chomeilist = list(map((lambda y: ''.join(y)), list(map(lambda x: list(x), minacho))))

In [81]:
mina_finedata = pd.DataFrame()

for i in range(0,116):
    if i != 39:
        k = mina_data[i].iloc[:21,0:3].set_axis(['世代','a','b'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k = k.fillna(0)
        k = k.astype({'a':str, 'b':str})
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)
        k['b'] = k['b'].str.replace(',','')
        k['b'] = k['b'].str.replace(r'0\.0','0', regex=True)
    else:
        k = mina_data[i].iloc[:21,0:2].set_axis(['世代','a'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)        
        k['b'] = '0'
        
    mina_finedata = pd.concat([mina_finedata, k])

mina_finedata = mina_finedata.astype({'a':int, 'b':int}).reset_index(drop=True) 

mina_finedata['人口'] = mina_finedata['a'] + mina_finedata['b']
#

for r in mina_finedata.itertuples():
    if not(((r.a == 0) and (r.b==r.人口)) or ((r.b==0) and (r.a==r.人口))):
        print(r.町丁目名, ' ', r.a, r.b)

mina_finedata = mina_finedata.drop(columns=['a','b'])[['町丁目名','世代','人口']]
mina_finedata = pd.merge(mina_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

mina_finedata['区']='港'

In [13]:
#mina_finedata.to_excel(prefix + dataname + '（港）.xlsx', index=False)

In [36]:
mina_data[39]

,年齢,総数,男,Unnamed: 0,女,Unnamed: 1,年齢.1,総数.1,Unnamed: 2,男.1,Unnamed: 3,女.1,Unnamed: 4
0,0~4歳,"1,133",598,NaN,535,NaN,0~4歳,NaN,62,NaN,33,NaN,29
1,5~9歳,979,496,NaN,483,NaN,5~9歳,NaN,69,NaN,34,NaN,35
2,10~14歳,834,433,NaN,401,NaN,10~14歳,NaN,20,NaN,10,NaN,10
3,15~19歳,558,279,NaN,279,NaN,15~19歳,NaN,19,NaN,9,NaN,10
4,20~24歳,461,241,NaN,220,NaN,20~24歳,NaN,27,NaN,12,NaN,15
5,25~29歳,791,358,NaN,433,NaN,25~29歳,NaN,67,NaN,30,NaN,37
6,30~34歳,"1,155",517,NaN,638,NaN,30~34歳,NaN,133,NaN,65,NaN,68
7,35~39歳,"1,431",663,NaN,768,NaN,35~39歳,NaN,168,NaN,89,NaN,79
8,40~44歳,"1,633",777,NaN,856,NaN,40~44歳,NaN,137,NaN,72,NaN,65
9,45~49歳,"1,678",849,NaN,829,NaN,45~49歳,NaN,109,NaN,63,NaN,46


## 5. 渋谷

In [82]:
shib_data = pd.read_excel(prefix + '渋谷区2022年1月_rev.xlsx')

shib_finedata = shib_data[['町丁名','年齢','人口']].rename(columns={'町丁名':'町丁目名'})

shib_finedata = pd.merge(shib_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

shib_finedata['区']='渋谷'

In [15]:
#shib_finedata.to_excel(prefix + dataname + '（渋谷）.xlsx', index=False)

### 台東

### データ統合

In [101]:
finedata = pd.concat([ohta_finedata,meg_finedata,shib_finedata,shina_finedata]).reset_index(drop=True)

finedata['世代'] = ''



In [102]:
finedata


,町丁目名,年齢,人口,大字町丁目名,大字町丁目コード,区,世代
0,大森東１丁目,0,24,大森東１丁目,131110039001,大田,
1,大森東１丁目,1,28,大森東１丁目,131110039001,大田,
2,大森東１丁目,2,36,大森東１丁目,131110039001,大田,
3,大森東１丁目,3,36,大森東１丁目,131110039001,大田,
4,大森東１丁目,4,31,大森東１丁目,131110039001,大田,
...,...,...,...,...,...,...,...
67584,東八潮,115,0,東八潮,131090005000,品川,
67585,東八潮,116,0,東八潮,131090005000,品川,
67586,東八潮,117,0,東八潮,131090005000,品川,
67587,東八潮,118,0,東八潮,131090005000,品川,


In [106]:
bin_band =  list(range(0,131,5))
#f=finedata[finedata['区']=='大田']
finedata['世代']=pd.cut(finedata['年齢'], bin_band, right=False, labels=fiveband)


In [108]:
finedata.loc[3000:3020,:]

,町丁目名,年齢,人口,大字町丁目名,大字町丁目コード,区,世代
3000,大森北５丁目,72,32,大森北５丁目,131110015005,大田,70~74歳
3001,大森北５丁目,73,32,大森北５丁目,131110015005,大田,70~74歳
3002,大森北５丁目,74,28,大森北５丁目,131110015005,大田,70~74歳
3003,大森北５丁目,75,27,大森北５丁目,131110015005,大田,75~79歳
3004,大森北５丁目,76,20,大森北５丁目,131110015005,大田,75~79歳
3005,大森北５丁目,77,15,大森北５丁目,131110015005,大田,75~79歳
3006,大森北５丁目,78,26,大森北５丁目,131110015005,大田,75~79歳
3007,大森北５丁目,79,17,大森北５丁目,131110015005,大田,75~79歳
3008,大森北５丁目,80,20,大森北５丁目,131110015005,大田,80~84歳
3009,大森北５丁目,81,13,大森北５丁目,131110015005,大田,80~84歳


In [56]:
bin_band

[0,
 5,
 10,
 15,
 20,
 25,
 30,
 35,
 40,
 45,
 50,
 55,
 60,
 65,
 70,
 75,
 80,
 85,
 90,
 95,
 100,
 105,
 110,
 115,
 120,
 125,
 130]

In [62]:
# fiveband = list(mina_finedata.loc[:20,:]['世代'])
#print(fiveband)
fiveband = ['0~4歳', '5~9歳', '10~14歳', '15~19歳', '20~24歳', '25~29歳', '30~34歳',
 '35~39歳', '40~44歳', '45~49歳', '50~54歳', '55~59歳', '60~64歳', '65~69歳',
 '70~74歳', '75~79歳', '80~84歳', '85~89歳', '90~94歳', '95~99歳', '100~104歳'
 '105~109歳', '110~114歳', '115~119歳', '120~124歳', '125~129歳', '130歳以上']
# ['0~4歳', '5~9歳', '10~14歳', '15~19歳', '20~24歳', '25~29歳', '30~34歳',
# '35~39歳', '40~44歳', '45~49歳', '50~54歳', '55~59歳', '60~64歳', '65~69歳',
# '70~74歳', '75~79歳', '80~84歳', '85~89歳', '90~94歳', '95~99歳', '100~104歳'
# '105~109歳', '110~114歳', '115~119歳', '120歳以上']

In [61]:
len(fiveband)

25